In [2]:
import os, numpy as np, tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import sys
from os import path
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [3]:
# Import file into colab path

if path.exists('full_file.py'):
  pass
else:
  for root, dirs, files in os.walk('PythonFiles'):
    with open("full_file.py", "w") as new_created_file:
      for f in files:
        with open('PythonFiles/'+f) as file:
          for line in file:
              new_created_file.write(line)
              
          new_created_file.write("\n")

In [4]:
# load python file that we create previouse step and covert to lowercase
filename = "full_file.py"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [5]:
print(':: Sample Code Extract ::')
print("="*25)
print(raw_text[1000:1500])

:: Sample Code Extract ::
ict the response
pred = knn.predict(x_test)

# evaluate accuracy
print("accuracy: {}".format(accuracy_score(y_test, pred)))

# creating odd list of k for knn
neighbors = list(range(1, 50, 2))

# empty list that will hold cv scores
cv_scores = []
crossvalidation = kfold(n_splits=10, shuffle=true, random_state=1)
# perform 10-fold cross validation
for k in neighbors:
    knn = kneighborsclassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train, cv=crossvalidation, scoring='accur


In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
# Convert characters to numeric (computer-readable form) > One Hot Encoding or Word Embedding
char_to_int = dict((c, i) for i, c in enumerate(chars))
# Convert numeric to categorical for showing prediction
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
# summarize the loaded data
# length of inputs
n_chars = len(raw_text) 
# number of unique characters
n_vocab = len(chars)

print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# set of characters
print(chars)

Total Characters:  693045
Total Vocab:  80
['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '°', '²', '×', 'å', 'ï', '’', '“', '”', '…']


In [8]:
# prepare the dataset of input to output pairs encoded as integers
# define the sequence length(length of input characters as integers)
seq_length = 100
dataX = []
dataY = []
# Convert computer-readable form
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 # input sequence of data
 dataX.append([char_to_int[char] for char in seq_in])
 #  our output
 dataY.append(char_to_int[seq_out])

# number of patterns
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  692945


In [9]:
# reshape X to be [samples, time steps, features]
# for fedding to the neural network
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# X.shape
# X[:10]

# apply one hot encoder to our output variable
y = to_categorical(dataY)
# y.shape
# dataY[:10]
# y[:10]

In [10]:
# implement LSTM model
# 3 Layer model with dropout to prevent overfitting
model = Sequential()
# initialized with 256 units of memory, return sequences of data rather than randomly scattered data
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
# output layer generate a probability about what next character 
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit our model with generated input and output
model.fit(X, y, epochs=15, batch_size=64, callbacks=callbacks_list)

Epoch 1/15
10827/10828 [============================>.] - ETA: 0s - loss: 2.7482
Epoch 1: loss improved from inf to 2.74823, saving model to weights-improvement-01-2.7482.hdf5
10828/10828 [==============================] - 216s 20ms/step - loss: 2.7482
Epoch 2/15
10826/10828 [============================>.] - ETA: 0s - loss: 2.2783
Epoch 2: loss improved from 2.74823 to 2.27833, saving model to weights-improvement-02-2.2783.hdf5
10828/10828 [==============================] - 212s 20ms/step - loss: 2.2783
Epoch 3/15
10828/10828 [==============================] - ETA: 0s - loss: 2.0547
Epoch 3: loss improved from 2.27833 to 2.05469, saving model to weights-improvement-03-2.0547.hdf5
10828/10828 [==============================] - 212s 20ms/step - loss: 2.0547
Epoch 4/15
10828/10828 [==============================] - ETA: 0s - loss: 1.9116
Epoch 4: loss improved from 2.05469 to 1.91165, saving model to weights-improvement-04-1.9116.hdf5
10828/10828 [==============================] - 211s 2

In [14]:
# load the network weights
filename = "weights-improvement-15-1.4471.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed to generate a sequence of character from
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Original Query:")
print("="*15)
# convert the output of the model from numbers to characters
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("*"*50)

# predict the characters
# involve converting the output numbers into characters and then append them to the pattern
print("\nPredicted Query:")
print("="*15)
for i in range(5000):
 x = np.reshape(pattern, (1, len(pattern), 1))
 x = x / float(n_vocab)
 prediction = model.predict(x, verbose=0)
 index = np.argmax(prediction)
 result = int_to_char[index]
 seq_in = [int_to_char[value] for value in pattern]
 print(result, end='')
#  sys.stdout.write(result)
 pattern.append(index)
 pattern = pattern[1:len(pattern)]
print("\nDone.")

Original Query:
" ain.mean()}')
print(f'test data scores saga : {lr2_test} mean: {lr2_test.mean()}')

lr1_pred1 = lr1_ "
**************************************************

Predicted Query:
clf_nestor.predict([some_digit3])
print('\n >> conf_sieels of cress in the coass iatr and the tertes tett acta the seruer corneet """

# - - mark # use the doass doassifier and the coass iath and the doame of the sesuert 

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           